# How to Download NFIP Datasets from OpenFEMA
Author: Mark Bauer

In [1]:
# import libraries
import logging
from datetime import datetime
from urllib.parse import urlencode
import requests
import duckdb
import pandas as pd

In [2]:
# reproducibility
%reload_ext watermark
%watermark -v -p requests,duckdb,pandas

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.6.0

requests: 2.28.1
duckdb  : 1.0.0
pandas  : 1.5.1



In [3]:
# data retrieved, not to be confused with data of data
current_date = datetime.now()
print(f"The data was retrieved on {current_date.strftime('%Y-%m-%d')}.")

The data was retrieved on 2025-05-31.


# Overview
This notebook demonstrates how to download and work with the National Flood Insurance Program (NFIP) [datasets](https://www.fema.gov/about/openfema/data-sets#nfip) from [OpenFEMA](https://www.fema.gov/about/reports-and-data/openfema). The workflows are designed to be efficient, easy to adapt, and easy to understand. These examples also include brief data exploration using [DuckDB](https://duckdb.org/).

## Datasets
- [FIMA NFIP Redacted Claims - v2](https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2)
- [FIMA NFIP Redacted Policies - v2](https://www.fema.gov/openfema-data-page/fima-nfip-redacted-policies-v2)
- [NFIP Multiple Loss Properties - v1](https://www.fema.gov/openfema-data-page/nfip-multiple-loss-properties-v1)
- [NFIP Residential Penetration Rates - v1](https://www.fema.gov/openfema-data-page/nfip-residential-penetration-rates-v1)
- [2023 NFIP Reinsurance Placement Information](https://www.fema.gov/about/openfema/data-sets/national-flood-insurance-program-nfip-reinsurance-placement-information)
- [NFIP Community Layer Comprehensive - v1](https://www.fema.gov/openfema-data-page/nfip-community-layer-comprehensive-v1)
- [NFIP Community Layer No Overlaps Split - v1](https://www.fema.gov/openfema-data-page/nfip-community-layer-no-overlaps-split-v1)
- [NFIP Community Layer No Overlaps Whole - v1](https://www.fema.gov/openfema-data-page/nfip-community-layer-no-overlaps-whole-v1)
- [NFIP Community Status Book - v1](https://www.fema.gov/openfema-data-page/nfip-community-status-book-v1)

## Notes
- Datasets are downloaded in Parquet format whenever possible for performance and compatibility.
- DuckDB is used for querying and previewing the data.
- The NFIP Community Layers dataset is large and has been excluded from this repository (tracked via .gitignore), but other datasets are included both locally and on GitHub.
- For the NFIP Policies and Claims datasets, the current focus is on a sample of New York City.

## Disclaimer
This analysis uses the Federal Emergency Management Agency’s OpenFEMA API, but is not endorsed by FEMA. The Federal Government or FEMA cannot vouch for the data or analyses derived from these data after the data have been retrieved from the Agency's website(s).

Read more about OpenFEMA's [Terms and Conditions](https://www.fema.gov/about/openfema/terms-conditions).

# Dataset Metadata
Before we get started, let's review the metadata for each dataset. Luckily, the OpenFEMA API has a great endpoint for this, `OpenFemaDataSets`.

In [4]:
url = (
    "https://www.fema.gov/api/open/v1/OpenFemaDataSets"
    "?$filter=theme eq 'National Flood Insurance Program'"
)

response = requests.get(url)

if response.ok:
    data = response.json()
    metadata_df = pd.DataFrame(data['OpenFemaDataSets'])
else:
    print(f"Failed to download data. Status code: {response.status_code}")
    
# preview data    
print(metadata_df.shape)    
metadata_df    

(8, 39)


,identifier,name,title,description,webService,dataDictionary,keyword,modified,publisher,contactPoint,...,recordCount,depDate,depApiMessage,depWebMessage,depNewURL,hash,lastRefresh,id,lastDataSetRefresh,distribution
0,openfema-86,NfipResidentialPenetrationRates,NFIP Residential Penetration Rates,"NFIP ""take-up rates"" or residential penetratio...",https://www.fema.gov/api/open/v1/NfipResidenti...,https://www.fema.gov/openfema-data-page/nfip-r...,"[nfip, rates, nsi, residential structures, pol...",2024-11-07T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,3162,None,None,None,None,c84151b7bbe46b75c5143a0c9d491a9986247f06,2024-11-21T14:40:15.168Z,18f6002e-40a5-4601-9504-279b8e6bf985,2025-05-19T15:20:16.607Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
1,openfema-78,NfipMultipleLossProperties,NFIP Multiple Loss Properties,This dataset provides information on structure...,https://www.fema.gov/api/open/v1/NfipMultipleL...,https://www.fema.gov/openfema-data-page/NFIP-M...,"[NFIP, Insurance, Flood, Repetitive Loss, Dama...",2024-02-27T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,257374,None,None,None,None,c6474d2bed656468b54afcb63f3da841959535a5,2024-11-21T14:40:15.168Z,f3cfd1cd-ea0d-455c-bfcd-f99625ecd393,2024-12-17T20:40:57.000Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
2,openfema-39,NfipCommunityStatusBook,NFIP Community Status Book,The National Flood Insurance Program (NFIP) en...,https://www.fema.gov/api/open/v1/NfipCommunity...,https://www.fema.gov/openfema-data-page/nfip-c...,"[NFIP, Status Book, Community Status Book, Flo...",2024-11-07T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,25053,None,None,None,None,7b8f6d05763d0c656c524c44438e1d8e8552abe8,2024-11-21T14:40:15.168Z,edccafe8-40f9-472a-84a9-e7cbbab850e2,2025-05-31T18:54:09.445Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
3,openfema-74,FimaNfipPolicies,FIMA NFIP Redacted Policies,Congress passed the National Flood Insurance A...,https://www.fema.gov/api/open/v2/FimaNfipPolicies,https://www.fema.gov/openfema-data-page/fima-n...,"[NFIP, Flood, Insurance, Policy, Water]",2025-04-01T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,69829956,None,None,None,None,85c6b91e237853b3333b06b4e3ebf5cdef930060,2025-04-01T20:00:37.246Z,3776bead-efe2-4590-84eb-3e000f869701,2025-05-14T13:43:49.278Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
4,openfema-77,NfipCommunityLayerComprehensive,NFIP Community Layer Comprehensive,This dataset is also known as the 3d layer and...,https://www.fema.gov/api/open/v1/NfipCommunity...,https://www.fema.gov/openfema-data-page/nfip-c...,"[nfip community, geospatial, census, boundaries]",2024-02-22T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,51943,None,None,None,None,51551de69709270cbb3dc05a9da88e92be099c2d,2024-12-05T22:20:23.322Z,0131d365-5a18-47a0-98cb-558149c165ee,2024-12-05T18:09:51.898Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
5,openfema-73,FimaNfipClaims,FIMA NFIP Redacted Claims,Congress passed the National Flood Insurance A...,https://www.fema.gov/api/open/v2/FimaNfipClaims,https://www.fema.gov/openfema-data-page/fima-n...,"[NFIP, FIMA, claims, flood insurance]",2023-03-24T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,2712269,None,None,None,None,843979427d2c4cc20e139ce661e6b0476975431f,2024-11-21T14:40:15.168Z,e9343760-d76e-44db-b947-822642353e7f,2025-05-14T13:01:34.036Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
6,openfema-76,NfipCommunityLayerNoOverlapsSplit,NFIP Community Layer No Overlaps Split,This dataset is flattened and multicounty comm...,https://www.fema.gov/api/open/v1/NfipCommunity...,https://www.fema.gov/openfema-data-page/nfip-c...,"[nfip community, geospatial, census, boundaries]",2024-02-22T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,...,37918,None,None,None,None,8db6fcd098f2b2bf16a3dc6b10ccb93543d32010,2024-12-05T22:20:23.322Z,57d33198-ea17-4255-83fe-35742b8f601b,2024-1

In [5]:
# first 15 columns
metadata_df.iloc[:, :15]

,identifier,name,title,description,webService,dataDictionary,keyword,modified,publisher,contactPoint,mbox,accessLevel,landingPage,temporal,api
0,openfema-86,NfipResidentialPenetrationRates,NFIP Residential Penetration Rates,"NFIP ""take-up rates"" or residential penetratio...",https://www.fema.gov/api/open/v1/NfipResidenti...,https://www.fema.gov/openfema-data-page/nfip-r...,"[nfip, rates, nsi, residential structures, pol...",2024-11-07T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,openfema@fema.gov,public,https://www.fema.gov/flood-insurance,None,True
1,openfema-78,NfipMultipleLossProperties,NFIP Multiple Loss Properties,This dataset provides information on structure...,https://www.fema.gov/api/open/v1/NfipMultipleL...,https://www.fema.gov/openfema-data-page/NFIP-M...,"[NFIP, Insurance, Flood, Repetitive Loss, Dama...",2024-02-27T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,openfema@fema.gov,public,https://www.fema.gov/flood-insurance,1978-01-01/,True
2,openfema-39,NfipCommunityStatusBook,NFIP Community Status Book,The National Flood Insurance Program (NFIP) en...,https://www.fema.gov/api/open/v1/NfipCommunity...,https://www.fema.gov/openfema-data-page/nfip-c...,"[NFIP, Status Book, Community Status Book, Flo...",2024-11-07T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,openfema@fema.gov,public,https://www.fema.gov/floodplain-management,1968-01-01/,True
3,openfema-74,FimaNfipPolicies,FIMA NFIP Redacted Policies,Congress passed the National Flood Insurance A...,https://www.fema.gov/api/open/v2/FimaNfipPolicies,https://www.fema.gov/openfema-data-page/fima-n...,"[NFIP, Flood, Insurance, Policy, Water]",2025-04-01T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,openfema@fema.gov,public,https://www.fema.gov/flood-insurance,2009-01-01/,True
4,openfema-77,NfipCommunityLayerComprehensive,NFIP Community Layer Comprehensive,This dataset is also known as the 3d layer and...,https://www.fema.gov/api/open/v1/NfipCommunity...,https://www.fema.gov/openfema-data-page/nfip-c...,"[nfip community, geospatial, census, boundaries]",2024-02-22T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,openfema@fema.gov,public,https://www.fema.gov/flood-insurance,None,True
5,openfema-73,FimaNfipClaims,FIMA NFIP Redacted Claims,Congress passed the National Flood Insurance A...,https://www.fema.gov/api/open/v2/FimaNfipClaims,https://www.fema.gov/openfema-data-page/fima-n...,"[NFIP, FIMA, claims, flood insurance]",2023-03-24T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,openfema@fema.gov,public,https://www.fema.gov/flood-insurance,1970-08-31/,True
6,openfema-76,NfipCommunityLayerNoOverlapsSplit,NFIP Community Layer No Overlaps Split,This dataset is flattened and multicounty comm...,https://www.fema.gov/api/open/v1/NfipCommunity...,https://www.fema.gov/openfema-data-page/nfip-c...,"[nfip community, geospatial, census, boundaries]",2024-02-22T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,openfema@fema.gov,public,https://www.fema.gov/flood-insurance,None,True
7,openfema-75,NfipCommunityLayerNoOverlapsWhole,NFIP Community Layer No Overlaps Whole,This dataset is flattened and multicounty comm...,https://www.fema.gov/api/open/v1/NfipCommunity...,https://www.fema.gov/openfema-data-page/nfip-c...,"[nfip community, geospatial, census, boundaries]",2024-02-22T00:00:00.000Z,Federal Emergency Management Agency,OpenFEMA,openfema@fema.gov,public,https://www.fema.gov/flood-insurance,None,True


In [6]:
# next 15 columns
metadata_df.iloc[:, 15:30]

,version,bureauCode,programCode,accessLevelComment,license,spatial,theme,dataQuality,accrualPeriodicity,language,primaryITInvestmentUII,references,issued,systemOfRecords,recordCount
0,1,[024:70],[024:039],None,None,None,National Flood Insurance Program,true,R/P3M,en-US,None,[],2024-11-08T00:00:00.000Z,None,3162
1,1,[024:70],[024:039],None,None,None,National Flood Insurance Program,true,R/P1Y,en-US,None,[https://agents.floodsmart.gov/sites/default/f...,2024-02-12T00:00:00.000Z,None,257374
2,1,[024:70],[024:039],None,None,None,National Flood Insurance Program,true,R/P1D,en-US,None,[],1968-01-01T00:00:00.000Z,None,25053
3,2,[024:70],[024:039],None,None,None,National Flood Insurance Program,true,R/P1M,en-US,None,[https://nfipservices.floodsmart.gov/manuals/j...,2019-06-01T00:00:00.000Z,None,69829956
4,1,[024:70],[024:039],None,None,None,National Flood Insurance Program,true,R/P1Y,en-US,None,[],2024-02-09T00:00:00.000Z,None,51943
5,2,[024:70],[024:039],None,None,None,National Flood Insurance Program,true,R/P1M,en-US,None,[https://nfipservices.floodsmart.gov/manuals/j...,2019-06-01T00:00:00.000Z,None,2712269
6,1,[024:70],[024:039],None,None,None,National Flood Insurance Program,true,R/P1Y,en-US,None,[],2024-02-09T00:00:00.000Z,None,37918
7,1,[024:70],[024:039],None,None,None,National Flood Insurance Program,true,R/P1Y,en-US,None,[],2024-02-09T00:00:00.000Z,None,29898


In [7]:
# columns to the end
metadata_df.iloc[:, 30:]

,depDate,depApiMessage,depWebMessage,depNewURL,hash,lastRefresh,id,lastDataSetRefresh,distribution
0,None,None,None,None,c84151b7bbe46b75c5143a0c9d491a9986247f06,2024-11-21T14:40:15.168Z,18f6002e-40a5-4601-9504-279b8e6bf985,2025-05-19T15:20:16.607Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
1,None,None,None,None,c6474d2bed656468b54afcb63f3da841959535a5,2024-11-21T14:40:15.168Z,f3cfd1cd-ea0d-455c-bfcd-f99625ecd393,2024-12-17T20:40:57.000Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
2,None,None,None,None,7b8f6d05763d0c656c524c44438e1d8e8552abe8,2024-11-21T14:40:15.168Z,edccafe8-40f9-472a-84a9-e7cbbab850e2,2025-05-31T18:54:09.445Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
3,None,None,None,None,85c6b91e237853b3333b06b4e3ebf5cdef930060,2025-04-01T20:00:37.246Z,3776bead-efe2-4590-84eb-3e000f869701,2025-05-14T13:43:49.278Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
4,None,None,None,None,51551de69709270cbb3dc05a9da88e92be099c2d,2024-12-05T22:20:23.322Z,0131d365-5a18-47a0-98cb-558149c165ee,2024-12-05T18:09:51.898Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
5,None,None,None,None,843979427d2c4cc20e139ce661e6b0476975431f,2024-11-21T14:40:15.168Z,e9343760-d76e-44db-b947-822642353e7f,2025-05-14T13:01:34.036Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
6,None,None,None,None,8db6fcd098f2b2bf16a3dc6b10ccb93543d32010,2024-12-05T22:20:23.322Z,57d33198-ea17-4255-83fe-35742b8f601b,2024-12-05T18:09:57.312Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."
7,None,None,None,None,7bdabebd0a0805adbb76fee3ca61bbd4a5f5c0b7,2024-12-05T22:20:23.322Z,e2c2268c-7a3c-4ce1-830e-ac5f735ee44c,2024-12-05T18:10:50.474Z,"[{'format': 'csv', 'accessURL': 'https://www.f..."


# Download Tutorial

Code to download the NFIP Policies and Claims datasets using the OpenFEMA API. In this tutorial, we only export data in New York City.

In [8]:
BASE_URL = "https://www.fema.gov/api/open/v2/FimaNfip{}?"
PAGE_SIZE = 10000

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

def build_url(dataset, fips, skip):
    params = {
        "$format": "json",
        "$filter": f"countyCode eq '{fips}'",
        "$top": PAGE_SIZE,
        "$skip": skip,
        "$metadata": "off",
    }
    return BASE_URL.format(dataset.capitalize()), params

def fetch_single_fips(dataset, fips):
    records = []
    skip = 0
    logging.info(f"Starting download for FIPS {fips}")

    while True:
        url, params = build_url(dataset, fips, skip)
        try:
            logging.info(f"Requesting: skip={skip}")
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()
            page = response.json().get(f"FimaNfip{dataset}", [])
            if not page:
                logging.info(f"No more data for FIPS {fips} at skip={skip}")
                break
            records.extend(page)
            if len(page) < PAGE_SIZE:
                logging.info(f"Final page for FIPS {fips} retrieved")
                break
            skip += PAGE_SIZE
        except requests.RequestException as e:
            logging.error(f"Error fetching data for FIPS {fips} at skip={skip}: {e}")
            break

    logging.info(f"Finished download for FIPS {fips}: {len(records)} records")
    return records

def fetch_nfip_data(dataset, fips_list, output_path):
    all_data = []
    for fips in fips_list:
        all_data.extend(fetch_single_fips(dataset, fips))

    if all_data:
        df = pd.DataFrame(all_data)
        df.to_parquet(output_path, index=False)
        logging.info(f"Saved {len(df)} records to {output_path}")
    else:
        logging.warning("No data fetched. Nothing to save.")

# OpenFEMA Dataset: FIMA NFIP Redacted Claims - v2
https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2

In [9]:
%%time

fips_list = [
    "36005",
    "36047",
    "36061",
    "36081",
    "36085"
]

output_path = "data/nyc-claims.parquet"

fetch_nfip_data("Claims", fips_list=fips_list, output_path=output_path)

2025-05-31 15:29:11,405 - INFO - Starting download for FIPS 36005
2025-05-31 15:29:11,406 - INFO - Requesting: skip=0
2025-05-31 15:29:14,405 - INFO - Final page for FIPS 36005 retrieved
2025-05-31 15:29:14,406 - INFO - Finished download for FIPS 36005: 1943 records
2025-05-31 15:29:14,406 - INFO - Starting download for FIPS 36047
2025-05-31 15:29:14,407 - INFO - Requesting: skip=0
2025-05-31 15:29:18,497 - INFO - Final page for FIPS 36047 retrieved
2025-05-31 15:29:18,498 - INFO - Finished download for FIPS 36047: 6263 records
2025-05-31 15:29:18,500 - INFO - Starting download for FIPS 36061
2025-05-31 15:29:18,500 - INFO - Requesting: skip=0
2025-05-31 15:29:20,285 - INFO - Final page for FIPS 36061 retrieved
2025-05-31 15:29:20,286 - INFO - Finished download for FIPS 36061: 1486 records
2025-05-31 15:29:20,287 - INFO - Starting download for FIPS 36081
2025-05-31 15:29:20,287 - INFO - Requesting: skip=0
2025-05-31 15:29:24,656 - INFO - Requesting: skip=10000
2025-05-31 15:29:27,558 -

CPU times: user 2.27 s, sys: 302 ms, total: 2.58 s
Wall time: 28.3 s


In [10]:
# count rows
duckdb.sql("""
    SELECT count(*) AS count_rows
    FROM read_parquet('data/nyc-claims.parquet')
""")

┌────────────┐
│ count_rows │
│   int64    │
├────────────┤
│      43979 │
└────────────┘

In [11]:
# preview first five rows
duckdb.sql("""
    SELECT *
    FROM read_parquet('data/nyc-claims.parquet')
    LIMIT 5
""").df()

,agricultureStructureIndicator,asOfDate,basementEnclosureCrawlspaceType,policyCount,crsClassificationCode,dateOfLoss,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,baseFloodElevation,...,rentalPropertyIndicator,state,reportedCity,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id
0,False,None,1.0,1,NaN,2021-09-01T00:00:00.000Z,False,None,NaN,NaN,...,False,NY,Currently Unavailable,10475,36005,36005045600,360050456003,40.9,-73.8,99d7829c-e79c-4b8b-9de7-af1863973b42
1,False,None,NaN,1,NaN,1984-03-08T00:00:00.000Z,False,None,NaN,NaN,...,False,NY,Currently Unavailable,10465,36005,36005027402,360050274023,40.8,-73.8,f4a3f994-52e8-4823-9a2d-58d9ea0d2577
2,False,None,NaN,1,NaN,1979-01-24T00:00:00.000Z,False,None,NaN,NaN,...,False,NY,Currently Unavailable,10463,36005,36005030100,360050301001,40.9,-73.9,99044017-de9b-4a48-8473-7d229d5fea7a
3,False,None,NaN,1,NaN,2012-10-29T00:00:00.000Z,False,None,NaN,NaN,...,False,NY,Currently Unavailable,10465,36005,36005013200,360050132003,40.8,-73.8,e848b356-c15b-4481-b498-598868ffc8bd
4,False,None,NaN,1,NaN,1992-12-11T00:00:00.000Z,False,None,NaN,NaN,...,False,NY,Currently Unavailable,10465,36005,36005013200,360050132001,40.8,-73.8,fc26b2e2-4ae8-4f0c-a27d-c8560631c901


In [12]:
# preview columns
duckdb.sql("""
    DESCRIBE (FROM read_parquet('data/nyc-claims.parquet'))
""").show(max_rows=100)

┌────────────────────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│                column_name                 │ column_type │  null   │   key   │ default │  extra  │
│                  varchar                   │   varchar   │ varchar │ varchar │ varchar │ varchar │
├────────────────────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ agricultureStructureIndicator              │ BOOLEAN     │ YES     │ NULL    │ NULL    │ NULL    │
│ asOfDate                                   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ basementEnclosureCrawlspaceType            │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ policyCount                                │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ crsClassificationCode                      │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ dateOfLoss                                 │ VARCHAR     │ YES     │ NULL    │ NULL    │ 

# OpenFEMA Dataset: FIMA NFIP Redacted Policies - v2
https://www.fema.gov/openfema-data-page/fima-nfip-redacted-policies-v2

In [13]:
%%time

fips_list = [
    "36005",
    "36047",
    "36061",
    "36081",
    "36085"
]

output_path = "data/nyc-policies.parquet"

fetch_nfip_data("Policies", fips_list=fips_list, output_path=output_path)

2025-05-31 15:29:39,793 - INFO - Starting download for FIPS 36005
2025-05-31 15:29:39,794 - INFO - Requesting: skip=0
2025-05-31 15:29:47,170 - INFO - Requesting: skip=10000
2025-05-31 15:29:53,903 - INFO - Requesting: skip=20000
2025-05-31 15:30:01,326 - INFO - Requesting: skip=30000
2025-05-31 15:30:06,200 - INFO - Final page for FIPS 36005 retrieved
2025-05-31 15:30:06,201 - INFO - Finished download for FIPS 36005: 35579 records
2025-05-31 15:30:06,204 - INFO - Starting download for FIPS 36047
2025-05-31 15:30:06,205 - INFO - Requesting: skip=0
2025-05-31 15:30:12,881 - INFO - Requesting: skip=10000
2025-05-31 15:30:20,467 - INFO - Requesting: skip=20000
2025-05-31 15:30:28,335 - INFO - Requesting: skip=30000
2025-05-31 15:30:36,298 - INFO - Requesting: skip=40000
2025-05-31 15:30:43,166 - INFO - Requesting: skip=50000
2025-05-31 15:30:49,867 - INFO - Requesting: skip=60000
2025-05-31 15:30:56,149 - INFO - Requesting: skip=70000
2025-05-31 15:31:02,575 - INFO - Requesting: skip=8000

CPU times: user 30.5 s, sys: 3.65 s, total: 34.1 s
Wall time: 6min 30s


In [14]:
# count rows
duckdb.sql("""
    SELECT count(*) AS count_rows
    FROM read_parquet('data/nyc-policies.parquet')
""")

┌────────────┐
│ count_rows │
│   int64    │
├────────────┤
│     553692 │
└────────────┘

In [15]:
# preview first five rows
duckdb.sql("""
    SELECT *
    FROM read_parquet('data/nyc-policies.parquet')
    LIMIT 5
""").df()

,agricultureStructureIndicator,baseFloodElevation,basementEnclosureCrawlspaceType,cancellationDateOfFloodPolicy,condominiumCoverageTypeCode,construction,crsClassCode,buildingDeductibleCode,contentsDeductibleCode,elevatedBuildingIndicator,...,femaRegion,propertyState,reportedCity,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id
0,False,NaN,NaN,None,N,False,NaN,0,0,False,...,2.0,NY,Currently Unavailable,10475,36005,36005030200,360050302003,40.9,-73.8,5aaca3b8-03f7-4a82-8636-5fc15c2886d1
1,False,NaN,1.0,None,N,False,NaN,5,5,False,...,2.0,NY,Currently Unavailable,10474,36005,36005009300,360050093001,40.8,-73.9,ad5de8ef-83f2-47fb-9fe9-d48ec7ac6068
2,False,13.0,0.0,None,N,False,NaN,0,0,False,...,2.0,NY,Currently Unavailable,10473,36005,36005000400,360050004004,40.8,-73.8,bbaa372c-423f-4144-b4dc-61c7a4fe1093
3,False,NaN,2.0,None,N,False,NaN,None,0,False,...,2.0,NY,Currently Unavailable,10462,36005,36005009200,360050092002,40.8,-73.8,06122838-e205-4f54-a7c2-3d92e03790c6
4,False,NaN,1.0,None,N,False,NaN,None,0,False,...,2.0,NY,Currently Unavailable,10465,36005,36005016000,360050160002,40.8,-73.8,c69f6b9c-14d3-44ac-8074-6757130e2414


In [16]:
# preview columns
duckdb.sql("""
    DESCRIBE (FROM read_parquet('data/nyc-policies.parquet'))
""").show(max_rows=100)

┌────────────────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│              column_name               │ column_type │  null   │   key   │ default │  extra  │
│                varchar                 │   varchar   │ varchar │ varchar │ varchar │ varchar │
├────────────────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ agricultureStructureIndicator          │ BOOLEAN     │ YES     │ NULL    │ NULL    │ NULL    │
│ baseFloodElevation                     │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ basementEnclosureCrawlspaceType        │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ cancellationDateOfFloodPolicy          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ condominiumCoverageTypeCode            │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ construction                           │ BOOLEAN     │ YES     │ NULL    │ NULL    │ NULL    │
│ crsClassCode                

# OpenFEMA Dataset: NFIP Multiple Loss Properties - v1
https://www.fema.gov/openfema-data-page/nfip-multiple-loss-properties-v1

In [17]:
%%time

# URL of the Parquet file to download
url = "https://www.fema.gov/api/open/v1/NfipMultipleLossProperties.parquet"

# path of saved file
file_path = "data/NfipMultipleLossProperties.parquet"

# send an HTTP GET request to the URL
response = requests.get(url)

# check if the request was successful (status code 200)
if response.status_code == 200:
    
    # write the content of the response to a file
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print("File downloaded successfully.")
    
else:
    print("Failed to download file.")

File downloaded successfully.
CPU times: user 324 ms, sys: 84.4 ms, total: 409 ms
Wall time: 55.3 s


In [18]:
# count rows
duckdb.sql("""
    SELECT count(*) AS count_rows
    FROM read_parquet('data/NfipMultipleLossProperties.parquet')
""")

┌────────────┐
│ count_rows │
│   int64    │
├────────────┤
│     257374 │
└────────────┘

In [19]:
# preview first five rows
duckdb.sql("""
    SELECT *
    FROM read_parquet('data/NfipMultipleLossProperties.parquet')
    LIMIT 5
""").df()

,fipsCountyCode,state,stateAbbreviation,county,zipCode,reportedCity,communityIdNumber,communityName,censusBlockGroup,nfipRl,...,occupancyType,originalConstructionDate,originalNBDate,postFIRMConstructionIndicator,primaryResidenceIndicator,mitigatedIndicator,insuredIndicator,totalLosses,mostRecentDateofLoss,id
0,01001,Alabama,AL,AUTAUGA COUNTY,36066,PRATTVILLE,010002,"PRATTVILLE, CITY OF",010010204001,True,...,11,1978-09-15,2023-06-07,True,False,False,True,2,2009-05-07,83afd8dc-c235-4f88-9870-095ea987764a
1,01001,Alabama,AL,AUTAUGA COUNTY,36067,PRATTVILLE,010002,"PRATTVILLE, CITY OF",010010207003,True,...,14,1989-01-01,2023-09-02,True,True,False,True,3,2024-01-09,c93a7608-09d4-482e-9ca7-799bf4e8f977
2,01001,Alabama,AL,AUTAUGA COUNTY,36067,PRATTVILLE,010002,"PRATTVILLE, CITY OF",010010202002,True,...,4,1935-01-01,2009-02-02,False,False,True,False,3,2009-05-07,0e19f3a6-2b31-4d94-b5fe-eec5985be6da
3,01001,Alabama,AL,AUTAUGA COUNTY,36066,PRATTVILLE,010002,"PRATTVILLE, CITY OF",010010204001,False,...,1,1976-07-01,1986-01-25,False,True,True,False,5,2000-09-01,1b298520-0bc9-484c-820b-3b86ba949caf
4,01001,Alabama,AL,AUTAUGA COUNTY,36067,PRATTVILLE,010002,"PRATTVILLE, CITY OF",010010207003,True,...,4,1978-01-01,1997-07-13,False,False,False,False,4,2000-09-01,1bba2518-673b-422a-a8d2-d2954d0b6c27


In [20]:
# preview columns
duckdb.sql("""
    DESCRIBE (FROM read_parquet('data/NfipMultipleLossProperties.parquet'))
""").show(max_rows=100)

┌───────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│          column_name          │ column_type │  null   │   key   │ default │  extra  │
│            varchar            │   varchar   │ varchar │ varchar │ varchar │ varchar │
├───────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ fipsCountyCode                │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ state                         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ stateAbbreviation             │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ county                        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ zipCode                       │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ reportedCity                  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ communityIdNumber             │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ communityName                 

# OpenFEMA Dataset: NFIP Residential Penetration Rates - v1
https://www.fema.gov/openfema-data-page/nfip-residential-penetration-rates-v1

In [21]:
%%time

# URL of the Parquet file to download
url = "https://www.fema.gov/api/open/v1/NfipResidentialPenetrationRates.parquet"

# path of saved file
file_path = "data/NfipResidentialPenetrationRates.parquet"

# send an HTTP GET request to the URL
response = requests.get(url)

# check if the request was successful (status code 200)
if response.status_code == 200:
    
    # write the content of the response to a file
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print("File downloaded successfully.")
    
else:
    print("Failed to download file.")  

File downloaded successfully.
CPU times: user 22.5 ms, sys: 6.62 ms, total: 29.2 ms
Wall time: 931 ms


In [22]:
# count rows
duckdb.sql("""
    SELECT count(*) AS count_rows
    FROM read_parquet('data/NfipResidentialPenetrationRates.parquet')
""")

┌────────────┐
│ count_rows │
│   int64    │
├────────────┤
│       3162 │
└────────────┘

In [23]:
# preview first five rows
duckdb.sql("""
    SELECT *
    FROM read_parquet('data/NfipResidentialPenetrationRates.parquet')
    LIMIT 5
""").df()

,state,county,resPenetrationRate,resContractsInForce,totalResStructures,fipsCode,asOfDate,id
0,Alabama,Autauga,0.0114,237,20789,1001,2025-05-15,95f9d9a4-298c-445a-b279-0cfa447d1850
1,Alabama,Baldwin,0.1078,10501,97390,1003,2025-05-15,41afe779-2ce5-43ee-bdd2-1b087261b01a
2,Alabama,Barbour,0.0031,32,10455,1005,2025-05-15,9d25a05c-bda0-4bf5-8e9e-55f712f53c9f
3,Alabama,Bibb,0.0027,21,7693,1007,2025-05-15,a4c74901-8061-4894-ae12-2ce0659888a8
4,Alabama,Blount,0.0018,40,22586,1009,2025-05-15,cb3af383-fb9e-4d7c-8f72-97b98342b871


In [24]:
# preview columns
duckdb.sql("""
    DESCRIBE (FROM read_parquet('data/NfipResidentialPenetrationRates.parquet'))
""")

┌─────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│     column_name     │ column_type │  null   │   key   │ default │  extra  │
│       varchar       │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ state               │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ county              │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ resPenetrationRate  │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
│ resContractsInForce │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ totalResStructures  │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ fipsCode            │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ asOfDate            │ DATE        │ YES     │ NULL    │ NULL    │ NULL    │
│ id                  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────────────┴─────────────┴─────────┴─────────┴───────

# 2023 NFIP Reinsurance Placement Information
https://www.fema.gov/about/openfema/data-sets/national-flood-insurance-program-nfip-reinsurance-placement-information

In [25]:
%%time

# URL of the Parquet file to download
url = "https://www.fema.gov/sites/default/files/documents/fema_nfip-reinsurance-placement-information_2023.zip"

# path of saved file
file_path = "data/fema_nfip-reinsurance-placement-information_2023.zip"

# send an HTTP GET request to the URL
response = requests.get(url)

# check if the request was successful (status code 200)
if response.status_code == 200:
    
    # write the content of the response to a file
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print("File downloaded successfully.")
    
else:
    print("Failed to download file.")

File downloaded successfully.
CPU times: user 36.1 ms, sys: 24.3 ms, total: 60.4 ms
Wall time: 750 ms


Preview files inside ZIP.

In [26]:
!unzip -l data/fema_nfip-reinsurance-placement-information_2023.zip

Archive:  data/fema_nfip-reinsurance-placement-information_2023.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
   288003  09-27-2023 12:55   fema_nfip-reinsurance-placement-information_2023/NFIP_2023_KRSpatialKat_ForMarkets_ControlTotals.pdf
   342686  09-27-2023 12:55   fema_nfip-reinsurance-placement-information_2023/NFIP_2023_KRSpatialKat_InlandFlood_xNATCPrecip_ForMarkets_ResultsSummary.pdf
   350586  09-27-2023 12:55   fema_nfip-reinsurance-placement-information_2023/NFIP_2023_KRSpatialKat_StormSurge_wNATCPrecip_ForMarkets_ResultsSummary.pdf
   388991  09-27-2023 12:55   fema_nfip-reinsurance-placement-information_2023/NFIP_2023_RMSv21_StormSurge_ForMarkets_ControlTotals.pdf
   667614  09-27-2023 12:55   fema_nfip-reinsurance-placement-information_2023/NFIP_2023_RMSv21_StormSurge_ForMarkets_ResultsSummary.pdf
   687022  09-27-2023 12:55   fema_nfip-reinsurance-placement-information_2023/NFIP_2023_VeriskTSv9_InlandFlood_ForMarkets_ResultsSummary.pdf
   68

# OpenFEMA Dataset: NFIP Community Layer Comprehensive - v1
https://www.fema.gov/openfema-data-page/nfip-community-layer-comprehensive-v1

In [27]:
%%time

# URL of the Parquet file to download
url = "https://www.fema.gov/api/open/v1/NfipCommunityLayerComprehensive.parquet"

# path of saved file
file_path = "data/NfipCommunityLayerComprehensive.parquet"

# send an HTTP GET request to the URL
response = requests.get(url)

# check if the request was successful (status code 200)
if response.status_code == 200:
    
    # write the content of the response to a file
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print("File downloaded successfully.")
    
else:
    print("Failed to download file.")

File downloaded successfully.
CPU times: user 4.78 s, sys: 1.85 s, total: 6.63 s
Wall time: 1min 50s


In [28]:
# count rows
duckdb.sql("""
    SELECT count(*) AS count_rows
    FROM read_parquet('data/NfipCommunityLayerComprehensive.parquet')
""")

┌────────────┐
│ count_rows │
│   int64    │
├────────────┤
│      51943 │
└────────────┘

In [29]:
# preview first five rows
duckdb.sql("""
    SELECT *
    FROM read_parquet('data/NfipCommunityLayerComprehensive.parquet')
    LIMIT 5
""").df()

,id,communityIdNumber,stateCode,countyCode,censusGeoid,county,communityName,communityNameShort,censusYear,censusPopulationEntire,...,cisSource,geometrySource,alternateGeoid,alternateName,alternateLongName,layerCreationNotes,censusClassCodes,censusFunctionalStatusCodes,layerTypeCode,layerGeometry
0,1,None,32,32013,3244000,Humboldt,None,None,2023,124,...,None,Census Funcstat S,None,McDermitt,McDermitt CDP,None,U1,None,Statistical Area,"{""type"":""MultiPolygon"",""coordinates"":[[[[-117...."
1,2,530297,None,None,None,None,"YAKAMA NATION, CONFEDERATED TRIBES AND BAND OF",YAKAMA NATION,2023,16902,...,PIVOT,NFHL,53077C_530297,"Yakama Nation, CONFEDERATED TRIBES AND BANDS,",None,None,None,None,TRIBE,"{""type"":""MultiPolygon"",""coordinates"":[[[[-120...."
2,3,050184,05,05123,05123,St. Francis,ST. FRANCIS COUNTY *,ST. FRANCIS*,2023,23090,...,PIVOT,Census County,None,St. Francis,St. Francis County,None,H1,A,County,"{""type"":""MultiPolygon"",""coordinates"":[[[[-90.4..."
3,4,400351,40,40059,4009850,Harper,"BUFFALO, TOWN OF",BUFFALO,2023,1039,...,PIVOT,Census Place,None,Buffalo,Buffalo town,None,C1,A,Place,"{""type"":""MultiPolygon"",""coordinates"":[[[[-99.6..."
4,5,450185,45,45087,45087,Union,UNION COUNTY*,UNION*,2023,27244,...,PIVOT,Census County,None,Union,Union County,None,H1,A,County,"{""type"":""MultiPolygon"",""coordinates"":[[[[-81.4..."


In [30]:
# preview columns
duckdb.sql("""
    DESCRIBE (FROM read_parquet('data/NfipCommunityLayerComprehensive.parquet'))
""")

┌─────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│         column_name         │ column_type │  null   │   key   │ default │  extra  │
│           varchar           │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ id                          │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ communityIdNumber           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ stateCode                   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ countyCode                  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ censusGeoid                 │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ county                      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ communityName               │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ communityNameShort          │ VARCHAR     │ YES     

# OpenFEMA Dataset: NFIP Community Layer No Overlaps Split - v1
https://www.fema.gov/openfema-data-page/nfip-community-layer-no-overlaps-split-v1

In [31]:
%%time

# URL of the Parquet file to download
url = "https://www.fema.gov/api/open/v1/NfipCommunityLayerNoOverlapsSplit.parquet"

# path of saved file
file_path = "data/NfipCommunityLayerNoOverlapsSplit.parquet"

# send an HTTP GET request to the URL
response = requests.get(url)

# check if the request was successful (status code 200)
if response.status_code == 200:
    
    # write the content of the response to a file
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print("File downloaded successfully.")
    
else:
    print("Failed to download file.")   

File downloaded successfully.
CPU times: user 5.03 s, sys: 1.64 s, total: 6.67 s
Wall time: 2min 34s


In [32]:
# count rows
duckdb.sql("""
    SELECT count(*) AS count_rows
    FROM read_parquet('data/NfipCommunityLayerNoOverlapsSplit.parquet')
""")

┌────────────┐
│ count_rows │
│   int64    │
├────────────┤
│      37918 │
└────────────┘

In [33]:
# preview first five rows
duckdb.sql("""
    SELECT *
    FROM read_parquet('data/NfipCommunityLayerNoOverlapsSplit.parquet')
    LIMIT 5
""").df()

,id,communityIdNumber,stateCode,countyCode,censusGeoid,county,communityName,communityNameShort,censusYear,censusPopulationEntire,...,cisSource,geometrySource,alternateGeoid,alternateName,alternateLongName,layerCreationNotes,censusClassCodes,censusFunctionalStatusCodes,layerTypeCode,layerGeometry
0,1,190053,19,19061,1903160,Dubuque,"ASBURY, CITY OF",ASBURY,2024,5946,...,PIVOT,Census Place,None,Asbury,Asbury city,None,C1,A,Place,"{""type"":""MultiPolygon"",""coordinates"":[[[[-90.8..."
1,2,480206,48,48135,4853388,Ector,"ODESSA, CITY OF",ODESSA,2024,110636,...,PIVOT,Census Place,None,Odessa,Odessa city,None,C1,A,Place (multi-county),"{""type"":""MultiPolygon"",""coordinates"":[[[[-102...."
2,3,040040,04,04013,0412000,Maricopa,"CHANDLER, CITY OF",CHANDLER,2024,276046,...,PIVOT,Census Place,None,Chandler,Chandler city,None,C1,A,Place,"{""type"":""MultiPolygon"",""coordinates"":[[[[-111...."
3,4,010200,01,01121,0174592,Talladega,"TALLADEGA, CITY OF",TALLADEGA,2024,15861,...,PIVOT,Census Place,None,Talladega,Talladega city,None,C1,A,Place,"{""type"":""MultiPolygon"",""coordinates"":[[[[-86.1..."
4,5,400211,40,40113,4065300,Osage,"SAND SPRINGS, CITY OF",SAND SPRINGS,2024,386,...,PIVOT,Census Place,None,Sand Springs,Sand Springs city,None,C1,A,Place (multi-county),"{""type"":""MultiPolygon"",""coordinates"":[[[[-96.2..."


In [34]:
# preview columns
duckdb.sql("""
    DESCRIBE (FROM read_parquet('data/NfipCommunityLayerNoOverlapsSplit.parquet'))
""").show(max_rows=100)

┌─────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│         column_name         │ column_type │  null   │   key   │ default │  extra  │
│           varchar           │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ id                          │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ communityIdNumber           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ stateCode                   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ countyCode                  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ censusGeoid                 │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ county                      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ communityName               │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ communityNameShort          │ VARCHAR     │ YES     

# OpenFEMA Dataset: NFIP Community Layer No Overlaps Whole - v1
https://www.fema.gov/openfema-data-page/nfip-community-layer-no-overlaps-whole-v1

In [35]:
%%time

# URL of the Parquet file to download
url = "https://www.fema.gov/api/open/v1/NfipCommunityLayerNoOverlapsWhole.parquet"

# path of saved file
file_path = "data/NfipCommunityLayerNoOverlapsWhole.parquet"

# send an HTTP GET request to the URL
response = requests.get(url)

# check if the request was successful (status code 200)
if response.status_code == 200:
    
    # write the content of the response to a file
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print("File downloaded successfully.")
    
else:
    print("Failed to download file.") 

File downloaded successfully.
CPU times: user 5.04 s, sys: 1.45 s, total: 6.49 s
Wall time: 1min 46s


In [36]:
# count rows
duckdb.sql("""
    SELECT count(*) AS count_rows
    FROM read_parquet('data/NfipCommunityLayerNoOverlapsWhole.parquet')
""")

┌────────────┐
│ count_rows │
│   int64    │
├────────────┤
│      29898 │
└────────────┘

In [37]:
# preview first five rows
duckdb.sql("""
    SELECT *
    FROM read_parquet('data/NfipCommunityLayerNoOverlapsWhole.parquet')
    LIMIT 5
""").df()

,id,communityIdNumber,stateCode,countyCode,censusGeoid,county,communityName,communityNameShort,censusYear,censusPopulationEntire,...,censusHousingUnitsPunched,landAreaPunched,cisType,cisSource,geometrySource,alternateName,censusClassCodes,censusFunctionalStatusCodes,layerTypeCode,layerGeometry
0,1,120213,12,12099,1239081,Palm Beach,"LAKE WORTH BEACH, CITY OF",LAKE WORTH BEACH,2024,42219,...,17476,17.677492,CITY,PIVOT,Census Place,Lake Worth Beach,C1,A,Place,"{""type"":""MultiPolygon"",""coordinates"":[[[[-80.0..."
1,2,120214,12,12099,1239375,Palm Beach,"LANTANA, TOWN OF",LANTANA,2024,11504,...,5659,7.708114,TOWN,PIVOT,Census Place,Lantana,C1,A,Place,"{""type"":""MultiPolygon"",""coordinates"":[[[[-80.0..."
2,3,120215,12,12099,1242700,Palm Beach,"MANALAPAN, TOWN OF",MANALAPAN,2024,419,...,306,27.319965,TOWN,PIVOT,Census Place,Manalapan,C1,A,Place,"{""type"":""MultiPolygon"",""coordinates"":[[[[-80.0..."
3,4,120216,12,12099,1242900,Palm Beach,"MANGONIA PARK, TOWN OF",MANGONIA PARK,2024,2142,...,750,1.941475,TOWN,PIVOT,Census Place,Mangonia Park,C1,A,Place,"{""type"":""MultiPolygon"",""coordinates"":[[[[-80.0..."
4,5,120217,12,12099,1249600,Palm Beach,"NORTH PALM BEACH, VILLAGE OF",NORTH PALM BEACH,2024,13162,...,8023,13.719528,VILLAGE,PIVOT,Census Place,North Palm Beach,C1,A,Place,"{""type"":""MultiPolygon"",""coordinates"":[[[[-80.0..."


In [38]:
# preview columns
duckdb.sql("""
    DESCRIBE (FROM read_parquet('data/NfipCommunityLayerNoOverlapsWhole.parquet'))
""")

┌─────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│         column_name         │ column_type │  null   │   key   │ default │  extra  │
│           varchar           │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ id                          │ INTEGER     │ YES     │ NULL    │ NULL    │ NULL    │
│ communityIdNumber           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ stateCode                   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ countyCode                  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ censusGeoid                 │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ county                      │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ communityName               │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ communityNameShort          │ VARCHAR     │ YES     

# OpenFEMA Dataset: NFIP Community Status Book - v1
https://www.fema.gov/openfema-data-page/nfip-community-status-book-v1

In [39]:
%%time

# URL of the Parquet file to download
url = "https://www.fema.gov/api/open/v1/NfipCommunityStatusBook.parquet"

# path of saved file
file_path = "data/NfipCommunityStatusBook.parquet"

# send an HTTP GET request to the URL
response = requests.get(url)

# check if the request was successful (status code 200)
if response.status_code == 200:
    
    # write the content of the response to a file
    with open(file_path, 'wb') as file:
        file.write(response.content)
    print("File downloaded successfully.")
    
else:
    print("Failed to download file.") 

File downloaded successfully.
CPU times: user 37.6 ms, sys: 25.8 ms, total: 63.4 ms
Wall time: 3.36 s


In [40]:
# count rows
duckdb.sql("""
    SELECT count(*) AS count_rows
    FROM read_parquet('data/NfipCommunityStatusBook.parquet')
""")

┌────────────┐
│ count_rows │
│   int64    │
├────────────┤
│      25053 │
└────────────┘

In [41]:
# preview first five rows
duckdb.sql("""
    SELECT *
    FROM read_parquet('data/NfipCommunityStatusBook.parquet')
    LIMIT 5
""").df()

,communityIdNumber,communityName,county,state,initialFloodHazardBoundaryMap,initialFloodInsuranceRateMap,currentlyEffectiveMapDate,regularEmergencyProgramDate,tribal,participatingInNFIP,originalEntryDate,classRatingEffectiveDate,classRating,sfhaDiscount,nonSfhaDiscount,lastRefresh
0,230563,"T06 R18 WELS, TOWNSHIP OF",SOMERSET COUNTY,ME,NaT,NaT,(NSFHA),04/30/84,False,True,NaT,NaT,None,None,None,2025-05-14 08:55:53.865
1,010232,"OHATCHEE, TOWN OF",CALHOUN COUNTY,AL,1975-01-31,2007-09-28,03/16/16,08/18/04,False,True,NaT,NaT,None,None,None,2025-05-14 08:54:44.047
2,010233,"CENTRE, CITY OF",CHEROKEE COUNTY,AL,1975-02-07,1980-03-14,03/16/16,03/14/80,False,True,NaT,NaT,None,None,None,2025-05-14 08:54:44.061
3,120275,"GULF BREEZE, CITY OF",SANTA ROSA COUNTY,FL,1974-06-28,1977-09-01,11/19/21,09/01/77,False,True,1993-10-01,2025-04-01,6,20%,10%,2025-05-14 08:55:06.066
4,120308,"EDGEWATER, CITY OF",VOLUSIA COUNTY,FL,1974-08-23,1980-09-03,09/29/17,09/03/80,False,True,1992-10-01,2025-04-01,6,20%,10%,2025-05-14 08:55:06.284


In [43]:
# preview columns
duckdb.sql("""
    DESCRIBE (FROM read_parquet('data/NfipCommunityStatusBook.parquet'))
""")

┌───────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│          column_name          │ column_type │  null   │   key   │ default │  extra  │
│            varchar            │   varchar   │ varchar │ varchar │ varchar │ varchar │
├───────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ communityIdNumber             │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ communityName                 │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ county                        │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ state                         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ initialFloodHazardBoundaryMap │ DATE        │ YES     │ NULL    │ NULL    │ NULL    │
│ initialFloodInsuranceRateMap  │ DATE        │ YES     │ NULL    │ NULL    │ NULL    │
│ currentlyEffectiveMapDate     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ regularEmergencyProgramDate   